2GZAV - SEMESTRÁLNÍ PROJEKT - Velčovský (2025) - 04 Aplikace malířských stylů OpenCV na video

In [2]:
import cv2
import sys
import numpy as np
import os

import matplotlib.pyplot as plt

from IPython.display import Image

In [9]:
PREVIEW  = 0 
MOSAIC   = 1  
CANDY    = 2
RAINPRINCESS = 3 
UDNIE    = 4

def get_model_path(styleName):
    styles = {
        "mosaic": "./models/weights/mosaic.onnx",
        "candy": "./models/weights/candy.onnx",
        "rain-princess": "./models/weights/rain-princess.onnx",
        "udnie": "./models/weights/udnie.onnx"
    }
    return styles.get(styleName, None)

def paint_styles(image, styleName):

    (h, w) = image.shape[:2]
    blob = cv2.dnn.blobFromImage(image, 1.0, (w, h),(103.939, 116.779, 123.680), swapRB=False, crop=False)

    # Načti model a proveď inferenci
    model_path = get_model_path(styleName)
    net = cv2.dnn.readNetFromONNX(model_path)
    net.setInput(blob)
    output = net.forward()
    output = output.squeeze().transpose((1, 2, 0))
    output += np.array([103.939, 116.779, 123.680])
    output = np.clip(output, 0, 255).astype(np.uint8)
    # output = cv2.cvtColor(output, cv2.COLOR_BGR2RGB)

    return output


image_filter = PREVIEW
alive = True

win_name = "Camera Paint styles"
cv2.namedWindow(win_name, cv2.WINDOW_NORMAL)
result = None

source = cv2.VideoCapture(0)

while alive:
    has_frame, frame = source.read()
    if not has_frame:
        break

    frame = cv2.flip(frame, 1)

    if image_filter == PREVIEW:
        result = frame
    elif image_filter == MOSAIC:
        result = paint_styles(frame, 'mosaic')
    elif image_filter == CANDY:
        result = paint_styles(frame, 'candy')
    elif image_filter == RAINPRINCESS:
        result = paint_styles(frame, 'rain-princess')
    elif image_filter == UDNIE:
        result = paint_styles(frame, 'udnie')

    cv2.imshow(win_name, result)

    key = cv2.waitKey(1)
    if key == ord("Q") or key == ord("q") or key == 27:
        alive = False
    elif key == ord("M") or key == ord("m"):
        image_filter = MOSAIC
    elif key == ord("C") or key == ord("c"):
        image_filter = CANDY
    elif key == ord("R") or key == ord("r"):
        image_filter = RAINPRINCESS
    elif key == ord("U") or key == ord("u"):
        image_filter = UDNIE

source.release()
cv2.destroyWindow(win_name)
